In [1]:
import torch
import torch.nn as nn

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
import sys
import time
import warnings
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib as mpl

import tensorflow as tf
from tensorflow import keras


torch.cuda.is_available()

True

In [2]:
x = torch.rand(5, 3)
print(x)

tensor([[0.1386, 0.6945, 0.8379],
        [0.0140, 0.0325, 0.2821],
        [0.3864, 0.8130, 0.2140],
        [0.6023, 0.2104, 0.5569],
        [0.6826, 0.2731, 0.4164]])


In [3]:

import os.path


train_all = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

NUMERIC_FEATURE_NAMES = [
    "Age",
    "RoomService",
    "FoodCourt",
    "ShoppingMall",
    "Spa",
    "VRDeck",
]

CATEGORICAL_FEATURE_NAMES1 = ["HomePlanet","CryoSleep","Destination","VIP"]


train_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [4]:
train_all.drop(['PassengerId', 'Cabin','Name','CryoSleep','VIP'],axis =1 , inplace=True)
test.drop(['PassengerId', 'Cabin','Name','CryoSleep','VIP'],axis = 1 ,inplace=True)
train_all.Transported.replace([True,False], ["1", "0"], inplace=True)

In [5]:
train_all["Age"].fillna(train_all["Age"].mean(skipna=True), inplace=True)
test["Age"].fillna(test["Age"].mean(skipna=True), inplace=True)

train_all["RoomService"].fillna(train_all["RoomService"].mean(skipna=True), inplace=True)
test["RoomService"].fillna(test["RoomService"].mean(skipna=True), inplace=True)

train_all["FoodCourt"].fillna(train_all["FoodCourt"].mean(skipna=True), inplace=True)
test["FoodCourt"].fillna(test["FoodCourt"].mean(skipna=True), inplace=True)

train_all["ShoppingMall"].fillna(train_all["ShoppingMall"].mean(skipna=True), inplace=True)
test["ShoppingMall"].fillna(test["ShoppingMall"].mean(skipna=True), inplace=True)

train_all["Spa"].fillna(train_all["Spa"].mean(skipna=True), inplace=True)
test["Spa"].fillna(test["Spa"].mean(skipna=True), inplace=True)

train_all["VRDeck"].fillna(train_all["VRDeck"].mean(skipna=True), inplace=True)
test["VRDeck"].fillna(test["VRDeck"].mean(skipna=True), inplace=True)


train_all["HomePlanet"].fillna('Kepler', inplace=True)
test["HomePlanet"].fillna('Kepler', inplace=True)

train_all["Destination"].fillna('TRAPPIST-1e', inplace=True)
test["Destination"].fillna('TRAPPIST-1e', inplace=True)

train_all.dropna(inplace = True)
test.dropna(inplace = True)

In [8]:
CSV_HEADER = [
    "HomePlanet",
    "Destination",
    "Age",
    "RoomService",
    "FoodCourt",
    "ShoppingMall",
    "Spa",
    "VRDeck",
    "Transported",
    ]

train_data_file = "train_data.csv"
test_data_file = "test_data.csv"

train_all.to_csv(train_data_file, index=False, header=False)
test.to_csv(test_data_file, index=False, header=False)

In [6]:
train, valid = train_test_split(train_all, test_size=0.2, random_state=42)

train_X, train_y = train.drop('Transported', axis=1), train['Transported']
valid_X, valid_y = valid.drop('Transported', axis=1), valid['Transported']
train_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6954 entries, 2333 to 7270
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    6954 non-null   object 
 1   Destination   6954 non-null   object 
 2   Age           6954 non-null   float64
 3   RoomService   6954 non-null   float64
 4   FoodCourt     6954 non-null   float64
 5   ShoppingMall  6954 non-null   float64
 6   Spa           6954 non-null   float64
 7   VRDeck        6954 non-null   float64
dtypes: float64(6), object(2)
memory usage: 489.0+ KB


In [7]:
train_X["PassengerId"]

KeyError: 'PassengerId'

In [ ]:

train_X["relation"] = train_X["PassengerId"].str.split("_").str[1].astype(int)
train_X["relation"]

2333    1
2589    2
8302    4
8177    2
500     2
       ..
5734    1
5191    1
5390    6
860     1
7270    1
Name: relation, Length: 6954, dtype: int64

In [ ]:
cat_encoder = OneHotEncoder()
housing_cat_encoded  = cat_encoder.fit_transform(train_X[["HomePlanet"]])
housing_cat_encoded.toarray()
train_X["HomePlanet"] = housing_cat_encoded.toarray()


In [ ]:
cat_encoder = OneHotEncoder()
housing_cat_encoded  = cat_encoder.fit_transform(train_X[["Destination"]])
housing_cat_encoded.toarray()
train_X["Destination"] = housing_cat_encoded.toarray()

In [ ]:
train_X["Deck"] = train_X["Cabin"].str[0]

train_X["Side"] = train_X["Cabin"].str[2]

train_X.drop(["Cabin", "Name"], axis=1, inplace=True)

cat_encoder = OneHotEncoder()
housing_cat_encoded  = cat_encoder.fit_transform(train_X[["Deck"]])
housing_cat_encoded.toarray()
train_X["Deck"] = housing_cat_encoded.toarray()

cat_encoder = OneHotEncoder()
housing_cat_encoded  = cat_encoder.fit_transform(train_X[["Side"]])
housing_cat_encoded.toarray()
train_X["Side"] = housing_cat_encoded.toarray()

train_X["Deck"]
train_X["Side"]

2333    0.0
2589    0.0
8302    0.0
8177    0.0
500     0.0
       ... 
5734    0.0
5191    0.0
5390    0.0
860     0.0
7270    0.0
Name: Side, Length: 6954, dtype: float64

In [ ]:
train_X

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,relation,Deck,Side
2333,2513_01,1.0,False,0.0,28.0,False,0.0,55.0,0.0,656.0,0.0,1,0.0,0.0
2589,2774_02,1.0,False,0.0,17.0,False,0.0,1195.0,31.0,0.0,0.0,2,0.0,0.0
8302,8862_04,0.0,True,1.0,28.0,False,0.0,0.0,0.0,0.0,0.0,4,0.0,0.0
8177,8736_02,0.0,False,0.0,20.0,False,NaN,2.0,289.0,976.0,0.0,2,0.0,0.0
500,0539_02,0.0,True,1.0,36.0,False,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,6076_01,1.0,NaN,0.0,18.0,False,14.0,2.0,144.0,610.0,0.0,1,0.0,0.0
5191,5537_01,0.0,False,0.0,50.0,NaN,690.0,0.0,30.0,762.0,428.0,1,0.0,0.0
5390,5756_06,1.0,False,0.0,22.0,False,158.0,0.0,476.0,0.0,26.0,6,0.0,0.0
860,0925_01,0.0,False,0.0,34.0,False,379.0,0.0,1626.0,0.0,0.0,1,0.0,0.0


In [ ]:
# train_X = np.asarray(train_X).astype('float32')
train_X

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,relation,Deck,Side
2333,2513_01,1.0,False,0.0,28.0,False,0.0,55.0,0.0,656.0,0.0,1,0.0,0.0
2589,2774_02,1.0,False,0.0,17.0,False,0.0,1195.0,31.0,0.0,0.0,2,0.0,0.0
8302,8862_04,0.0,True,1.0,28.0,False,0.0,0.0,0.0,0.0,0.0,4,0.0,0.0
8177,8736_02,0.0,False,0.0,20.0,False,NaN,2.0,289.0,976.0,0.0,2,0.0,0.0
500,0539_02,0.0,True,1.0,36.0,False,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,6076_01,1.0,NaN,0.0,18.0,False,14.0,2.0,144.0,610.0,0.0,1,0.0,0.0
5191,5537_01,0.0,False,0.0,50.0,NaN,690.0,0.0,30.0,762.0,428.0,1,0.0,0.0
5390,5756_06,1.0,False,0.0,22.0,False,158.0,0.0,476.0,0.0,26.0,6,0.0,0.0
860,0925_01,0.0,False,0.0,34.0,False,379.0,0.0,1626.0,0.0,0.0,1,0.0,0.0


In [ ]:
# model = keras.Sequential([
#     keras.layers.Dense(10, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')
# ])

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# model.fit(train_X, train_y, validation_data=(valid_X, valid_y))